# VLNCE+EnvDrop AUG
VLNCE 202779

VLNCE + AUG 601038 

In [1]:
import gc
import os,sys
os.chdir("/root/EvoEnc/")
import random
import warnings
from collections import defaultdict
from datetime import datetime
os.environ["MAGNUM_LOG"] = "quiet"
os.environ["HABITAT_SIM_LOG"] = "quiet"
import numpy as np
from PIL import Image
import copy
import torch
from pathlib import Path
from tqdm.notebook import tqdm
from habitat import logger
from habitat_baselines.common.baseline_registry import baseline_registry
from habitat_baselines.common.environments import get_env_class
from habitat_baselines.common.obs_transformers import (
    apply_obs_transforms_batch,
    apply_obs_transforms_obs_space,
    get_active_obs_transforms,
)
from habitat_baselines.common.tensorboard_utils import TensorboardWriter
from habitat_baselines.utils.common import batch_obs

from evoenc.common.aux_losses import AuxLosses
from evoenc.common.base_il_trainer import BaseVLNCETrainer
from evoenc.common.env_utils import construct_envs
from evoenc.common.utils import extract_instruction_tokens
from evoenc.config.default import get_config
def _pause_envs(
    envs_to_pause,
    envs,
    not_done_masks,
    prev_actions,
    batch,
    rgb_frames=None,
):
    # pausing envs with no new episode
    if len(envs_to_pause) > 0:
        state_index = list(range(envs.num_envs))
        for idx in reversed(envs_to_pause):
            state_index.pop(idx)
            envs.pause_at(idx)

        # indexing along the batch dimensions
        not_done_masks = not_done_masks[state_index]
        prev_actions = prev_actions[state_index]

        for k, v in batch.items():
            batch[k] = v[state_index]

        if rgb_frames is not None:
            rgb_frames = [rgb_frames[i] for i in state_index]

    return (
        envs,
        not_done_masks,
        prev_actions,
        batch,
        rgb_frames,
    )

/root/habitat-lab21/habitat/utils/visualizations/maps.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  AGENT_SPRITE = imageio.imread(
/root/miniconda3/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
2024-03-03 23:17:24.741244: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-03 23:17:24.797675: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use 

## VLN-CE train

In [2]:
CONFIG = "/root/EvoEnc/evoenc/config/old/evoenc_s2_tune.yaml"
FOLDER_RGB_OUT = Path("/root/stage3/rgb-vlnce/train")
FOLDER_DEPTH_OUT = Path("/root/stage3/rgb-vlnce/train")
FOLDER_INST_OUT = Path("/root/stage3/rgb-vlnce/train")
FOLDER_SUB_OUT = Path("/root/stage3/rgb-vlnce/train")

os.makedirs(FOLDER_RGB_OUT, exist_ok=True)
os.makedirs(FOLDER_DEPTH_OUT, exist_ok=True)
os.makedirs(FOLDER_INST_OUT, exist_ok=True)
os.makedirs(FOLDER_SUB_OUT, exist_ok=True)
total_cnt = 0

device = torch.device("cuda")
config = get_config(CONFIG)
envs = construct_envs(config, get_env_class(config.ENV_NAME))
expert_uuid = config.IL.DAGGER.expert_policy_sensor_uuid
prev_actions = torch.zeros(
    envs.num_envs,
    1,
    device=device,
    dtype=torch.long,
)
not_done_masks = torch.zeros(
    envs.num_envs, 1, dtype=torch.bool, device=device
)

observations = envs.reset()
observations_raw = copy.deepcopy(observations)
observations = extract_instruction_tokens(
    observations, config.TASK_CONFIG.TASK.INSTRUCTION_SENSOR_UUID
)
batch = batch_obs(observations, device)
obs_transforms = get_active_obs_transforms(config)
batch = apply_obs_transforms_batch(batch, obs_transforms)

episodes = [[] for _ in range(envs.num_envs)]
skips = [False for _ in range(envs.num_envs)]
# Populate dones with False initially
dones = [False for _ in range(envs.num_envs)]

collected_eps = 0
ep_ids_collected = None
ensure_unique_episodes = True
if ensure_unique_episodes:
    ep_ids_collected = {
        ep.episode_id for ep in envs.current_episodes()
    }
RGB_SIZE = 224
DEPTH_SIZE = 224
LEN = 256
PAD_IDX = 1
SUB_PAD_IDX = 1
SUB_LEN = 50
SUB_NUM = 12
DOWNSAMPLE = 2
with tqdm(total=config.IL.DAGGER.update_size, dynamic_ncols=True) as pbar:
    while collected_eps < config.IL.DAGGER.update_size:
        current_episodes = None
        envs_to_pause = None
        if ensure_unique_episodes:
            envs_to_pause = []
            current_episodes = envs.current_episodes()

        for i in range(envs.num_envs):
            if dones[i] and not skips[i]:
                ep = episodes[i]
                N = len(ep)
                for step in range(0, N, DOWNSAMPLE):
                    rgb = ep[step][0]["rgb"]
                    depth = ep[step][0]["depth"][..., 0]
                    text = ep[step][0]["instruction"]["text"]
                    sub = ep[step][0]["sub_instruction"]["text"]

                    rgb_path = FOLDER_RGB_OUT / "{}.jpg".format(total_cnt)
                    depth_path = FOLDER_DEPTH_OUT / "{}.png".format(total_cnt)
                    inst_path = FOLDER_INST_OUT / ("%08d.txt"%(total_cnt))
                    sub_path = FOLDER_SUB_OUT / ("%08d.txt"%(total_cnt))

                    rgb = Image.fromarray(rgb)
                    depth = (depth*10.0*1000).astype(np.uint16) # to meters, and scale 1000 times
                    depth = Image.fromarray(depth)
                    depth = depth.resize((DEPTH_SIZE, DEPTH_SIZE))

                    rgb.save(rgb_path)
                    depth.save(depth_path)
                    with open(inst_path, "w") as f:
                        f.write(text)
                    with open(sub_path, "w") as f:
                        f.write("\n".join(sub))

                    total_cnt+=1
                    

                pbar.update()
                collected_eps += 1

                if ensure_unique_episodes:
                    if (
                        current_episodes[i].episode_id
                        in ep_ids_collected
                    ):
                        envs_to_pause.append(i)
                    else:
                        ep_ids_collected.add(
                            current_episodes[i].episode_id
                        )
            if dones[i]:
                episodes[i] = []

        if ensure_unique_episodes:
            (
                envs,
                not_done_masks,
                prev_actions,
                batch,
                _,
            ) = _pause_envs(
                envs_to_pause,
                envs,
                not_done_masks,
                prev_actions,
                batch,
            )
            if envs.num_envs == 0:
                break

        actions = batch[expert_uuid].long()

        for i in range(envs.num_envs):
            episodes[i].append(
                (
                    observations_raw[i],
                    prev_actions[i].item(),
                    batch[expert_uuid][i].item(),
                )
            )

        skips = batch[expert_uuid].long() == -1
        actions = torch.where(
            skips, torch.zeros_like(actions), actions
        )
        skips = skips.squeeze(-1)  # .to(device="cpu")
        prev_actions.copy_(actions)

        outputs = envs.step([a[0].item() for a in actions])
        observations, _, dones, _ = [list(x) for x in zip(*outputs)]
        observations_raw = copy.deepcopy(observations)
        observations = extract_instruction_tokens(
            observations,
            config.TASK_CONFIG.TASK.INSTRUCTION_SENSOR_UUID,
        )
        batch = batch_obs(observations, device)
        batch = apply_obs_transforms_batch(batch, obs_transforms)

        not_done_masks = torch.tensor(
            [[0] if done else [1] for done in dones],
            dtype=torch.uint8,
            device=device,
        )


envs.close()
envs = None


2024-03-03 23:17:27,610 Initializing dataset VLN-CE-v1


/root/habitat-lab21/habitat/utils/visualizations/maps.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  AGENT_SPRITE = imageio.imread(
/root/habitat-lab21/habitat/utils/visualizations/maps.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  AGENT_SPRITE = imageio.imread(
/root/habitat-lab21/habitat/utils/visualizations/maps.py:25: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  AG

  0%|          | 0/5000 [00:00<?, ?it/s]

I0303 23:28:31.406900 924277 PhysicsManager.cpp:50] Deconstructing PhysicsManager
I0303 23:28:31.407104 924277 SemanticScene.h:43] Deconstructing SemanticScene
I0303 23:28:31.418263 924277 SceneManager.h:25] Deconstructing SceneManager
I0303 23:28:31.418285 924277 SceneGraph.h:25] Deconstructing SceneGraph
I0303 23:28:31.419302 924277 Sensor.cpp:69] Deconstructing Sensor
I0303 23:28:31.419404 924277 Sensor.cpp:69] Deconstructing Sensor
I0303 23:28:31.432915 924277 Renderer.cpp:71] Deconstructing Renderer
I0303 23:28:31.432942 924277 WindowlessContext.h:17] Deconstructing WindowlessContext
I0303 23:28:31.441830 924277 MetadataMediator.cpp:68] ::setSimulatorConfiguration : Set new simulator config for scene/stage : data/scene_datasets/mp3d/VFuaQ6m2Qom/VFuaQ6m2Qom.glb and dataset : default which is currently active dataset.
I0303 23:28:31.449214 924277 ManagedFileBasedContainer.h:210] <Scene Instance>::convertFilenameToPassedExt : Filename : data/scene_datasets/mp3d/VFuaQ6m2Qom/VFuaQ6m2Qo

In [3]:
episodes[0][0][0]["sub_instruction"]["text"]

['Exit the bathroom',
 'and turn left',
 'Pass the bed',
 'and pass the yellow couch',
 'Go into the door that has two candles on each side',
 'and wait by the towels on the wooden piece of furniture']

## AUG

In [ ]:
CONFIG = "/root/MLA/vlnce_baselines/config/transformer/transformer_aug.yaml"

device = torch.device("cuda")
config = get_config(CONFIG)
envs = construct_envs(config, get_env_class(config.ENV_NAME))
expert_uuid = config.IL.DAGGER.expert_policy_sensor_uuid
prev_actions = torch.zeros(
    envs.num_envs,
    1,
    device=device,
    dtype=torch.long,
)
not_done_masks = torch.zeros(
    envs.num_envs, 1, dtype=torch.bool, device=device
)

observations = envs.reset()
observations = extract_instruction_tokens(
    observations, config.TASK_CONFIG.TASK.INSTRUCTION_SENSOR_UUID
)
batch = batch_obs(observations, device)
obs_transforms = get_active_obs_transforms(config)
batch = apply_obs_transforms_batch(batch, obs_transforms)

episodes = [[] for _ in range(envs.num_envs)]
skips = [False for _ in range(envs.num_envs)]
# Populate dones with False initially
dones = [False for _ in range(envs.num_envs)]

collected_eps = 0
ep_ids_collected = None
ensure_unique_episodes = True
if ensure_unique_episodes:
    ep_ids_collected = {
        ep.episode_id for ep in envs.current_episodes()
    }

In [ ]:
RGB_SIZE = 224
DEPTH_SIZE = 256
SUB_LEN = 12
INST_LEN = 77
DOWNSAMPLE = 10
with tqdm(total=config.IL.DAGGER.update_size, dynamic_ncols=True) as pbar, h5py.File('/hy-tmp/stage2/data.mat','a') as t:
    target_rgb = t["rgb"]
    target_depth = t["depth"]
    target_inst = t["instructions"]
    target_sub = t["sub_instructions"]
    while collected_eps < config.IL.DAGGER.update_size and target_rgb.shape[0]<601034:
        current_episodes = None
        envs_to_pause = None
        if ensure_unique_episodes:
            envs_to_pause = []
            current_episodes = envs.current_episodes()

        for i in range(envs.num_envs):
            if dones[i] and not skips[i]:
                ep = episodes[i]
                traj_obs = batch_obs(
                    [step[0] for step in ep],
                    device=torch.device("cpu"),
                )
                for k, v in traj_obs.items():
                    traj_obs[k] = v.numpy()
                    if config.IL.DAGGER.lmdb_fp16:
                        traj_obs[k] = traj_obs[k].astype(np.float16)

                rgbi = traj_obs["rgb"][::DOWNSAMPLE]
                depthi = traj_obs["depth"][::DOWNSAMPLE]
                insti = traj_obs["instruction"][::DOWNSAMPLE]
                subi = traj_obs["sub_instruction"][::DOWNSAMPLE]
                n = rgbi.shape[0]
                assert (depthi.shape[0]==n and insti.shape[0]==n and subi.shape[0]==n)
                target_rgb.resize(target_rgb.shape[0]+n, axis=0)
                target_rgb[-n:] = rgbi
                target_depth.resize(target_depth.shape[0]+n, axis=0)
                target_depth[-n:] = depthi
                target_inst.resize(target_inst.shape[0]+n, axis=0)
                target_inst[-n:] = insti
                target_sub.resize(target_sub.shape[0]+n, axis=0)
                target_sub[-n:] = subi

                pbar.update()
                collected_eps += 1

                if ensure_unique_episodes:
                    if (
                        current_episodes[i].episode_id
                        in ep_ids_collected
                    ):
                        envs_to_pause.append(i)
                    else:
                        ep_ids_collected.add(
                            current_episodes[i].episode_id
                        )
            if dones[i]:
                episodes[i] = []

        if ensure_unique_episodes:
            (
                envs,
                not_done_masks,
                prev_actions,
                batch,
                _,
            ) = _pause_envs(
                envs_to_pause,
                envs,
                not_done_masks,
                prev_actions,
                batch,
            )
            if envs.num_envs == 0:
                break

        actions = batch[expert_uuid].long()

        for i in range(envs.num_envs):
            episodes[i].append(
                (
                    observations[i],
                    prev_actions[i].item(),
                    batch[expert_uuid][i].item(),
                )
            )

        skips = batch[expert_uuid].long() == -1
        actions = torch.where(
            skips, torch.zeros_like(actions), actions
        )
        skips = skips.squeeze(-1)  # .to(device="cpu")
        prev_actions.copy_(actions)

        outputs = envs.step([a[0].item() for a in actions])
        observations, _, dones, _ = [list(x) for x in zip(*outputs)]

        observations = extract_instruction_tokens(
            observations,
            config.TASK_CONFIG.TASK.INSTRUCTION_SENSOR_UUID,
        )
        batch = batch_obs(observations, device)
        batch = apply_obs_transforms_batch(batch, obs_transforms)

        not_done_masks = torch.tensor(
            [[0] if done else [1] for done in dones],
            dtype=torch.uint8,
            device=device,
        )


envs.close()
envs = None


In [ ]:
import gzip
import json
with gzip.open("/root/MLA/data/datasets/R2R_VLNCE_NRSub_T/train/train_sub.json.gz", "r") as f:
    data = json.loads(f.read())

In [ ]:
import gzip
import json
with gzip.open("/root/MLA/data/datasets/R2R_VLNCE_NRSub_T/joint_train_envdrop/joint_train_envdrop_sub.json.gz", "r") as f:
    aug_data = json.loads(f.read())

In [ ]:
with h5py.File('/hy-tmp/stage2/data.mat','r') as t:
    print(t["rgb"].shape)